In [1]:
!pip install scikit-surprise

Looking in indexes: https://artifactory.dataxu.net/artifactory/api/pypi/pypi-virtual/simple/


In [3]:
import surprise
import pandas as pd


In [4]:
df = pd.read_csv('movies_small.csv')

In [5]:
df.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres
0,1626,132,260,2.0,1157920586,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
1,1647,198,260,5.0,1034137711,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
2,1655,212,260,2.5,1505400094,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
3,1667,234,260,5.0,1004407958,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
4,1684,279,260,5.0,1506394310,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


In [8]:
reader = surprise.Reader(rating_scale=(1, 5))
data = surprise.Dataset.load_from_df(df[['userId','movieId','rating']],reader)

In [20]:
from surprise.prediction_algorithms.knns import KNNBasic
sim_options = {'name': 'cosine',
               'user_based': False
               }
algo_knn_basic = KNNBasic(sim_options=sim_options)
full_training_ds = data.build_full_trainset()


In [21]:
full_training_ds

In [22]:
algo_knn_basic.fit(full_training_ds)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [23]:
algo_knn_basic.predict(599,260)

Prediction(uid=599, iid=260, r_ui=None, est=2.9382726122616125, details={'actual_k': 36, 'was_impossible': False})

In [24]:
all_movies = pd.unique(df['movieId'])
all_movies

array([   260,    316,   1210,   1282,   2054,   2858,    914,    368,
         1907,   2019,    410,      7,     10,     65,    314,    355,
          426,    991,   1101,   1584,   1784,   5989,  72998,   1370,
         1608,   2694,   4011,   1299,   3039,   2422,  91542,    325,
          788,   2915,  81591,  57368,  61024,   5292,    605,   1672,
         2574,   4823,  42738,  75805, 166461,   3287,   6816])

In [25]:
rating_movies= []
for movie in all_movies:
    rating_estimado = algo_knn_basic.predict(599,movie)
    rating_movies.append((movie,rating_estimado.est))
rating_movies


[(260, 2.9382726122616125),
 (316, 2.929619178065219),
 (1210, 2.938469643989169),
 (1282, 2.9301436529211826),
 (2054, 2.933536902790784),
 (2858, 2.937321635749173),
 (914, 3.0120693562865455),
 (368, 2.928221325842058),
 (1907, 2.9576849979382374),
 (2019, 2.935264791730318),
 (410, 2.9299178214616317),
 (7, 2.9305299185290963),
 (10, 2.931493786745792),
 (65, 2.931030928461788),
 (314, 2.931305170621058),
 (355, 2.916779018807287),
 (426, 3.03785294155227),
 (991, 2.9290080490975248),
 (1101, 2.9316100117739197),
 (1584, 2.928389185433414),
 (1784, 2.936171685274034),
 (5989, 2.930704412221085),
 (72998, 2.925233107832978),
 (1370, 2.931460323901631),
 (1608, 2.9318189848152576),
 (2694, 2.9339942975568207),
 (4011, 2.9358836811925575),
 (1299, 2.9301406644073804),
 (3039, 2.9325637599938834),
 (2422, 2.9329699234655986),
 (91542, 2.9332432300574047),
 (325, 2.929246434939349),
 (788, 2.929897754059986),
 (2915, 2.9362428464381365),
 (81591, 2.9344007936117618),
 (57368, 2.93806365

In [26]:
df_recommend_movies = pd.DataFrame(rating_movies, columns =['movieId','rating_esperado'])
df_recommend_movies

,movieId,rating_esperado
0,260,2.938273
1,316,2.929619
2,1210,2.938470
3,1282,2.930144
4,2054,2.933537
5,2858,2.937322
6,914,3.012069
7,368,2.928221
8,1907,2.957685
9,2019,2.935265


In [27]:
df_recommend_movies.merge(df).groupby('movieId').max()[['title','rating_esperado']].sort_values('rating_esperado', ascending=False)

,title,rating_esperado
movieId,,
166461,Moana (2016),4.083333
2574,"Out-of-Towners, The (1999)",3.133333
75805,"Bounty Hunter, The (2010)",3.104167
426,Body Snatchers (1993),3.037853
914,My Fair Lady (1964),3.012069
6816,Three O'Clock High (1987),2.984848
5292,Slap Shot (1977),2.984848
1907,Mulan (1998),2.957685
3287,"Tigger Movie, The (2000)",2.944444


In [28]:
from surprise.model_selection.split import train_test_split
trainset, testset = train_test_split(data, test_size=0.25)